## 分组

分组通常是一个SAC过程  
SAC指的是分组操作中的split-apply-combine过程
其中split指基于某一些规则，将数据拆成若干组，apply是指对每一组独立地使用函数，combine指将每一组的结果组合成某一类数据结构

在apply过程中，我们实际往往会遇到四类问题：  
整合（Aggregation）——即分组计算统计量（如求均值、求每组元素个数）

变换（Transformation）——即分组对每个单元的数据进行操作（如元素标准化）

过滤（Filtration）——即按照某些规则筛选出一些组（如选出组内某一指标小于50的组）

综合问题——即前面提及的三种问题的混合

### groupBy（SAC中的S）
https://www.yiibai.com/pandas/python_pandas_groupby.html

In [119]:
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', np.nan, 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}

df = pd.DataFrame(ipl_data)
df

#经过groupby后会生成一个groupby对象，该对象本身不会返回任何东西，只有当相应的方法被调用才会起作用
group_year =  df.groupby('Year')

#对于groupby函数而言，分组的依据是非常自由的，只要是与数据框长度相同的列表即可，同时支持函数型分组
np.random.choice(['a','b','c'],df.shape[0])
df.groupby(np.random.choice(['a','b','c'],df.shape[0])).get_group('a')

#函数型分组
df.groupby(lambda x:'奇数行' if df.index.get_loc(x)%2==1 else '偶数行').get_group('奇数行')

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,NaN,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


array(['b', 'b', 'b', 'c', 'a', 'b', 'c', 'b', 'c', 'c', 'b', 'b'],
      dtype='<U1')

,Team,Rank,Year,Points
1,Riders,2,2015,789
7,Kings,1,2017,788
11,Riders,2,2017,690


,Team,Rank,Year,Points
1,Riders,2,2015,789
3,Devils,3,2015,673
5,kings,4,2015,812
7,Kings,1,2017,788
9,Royals,4,2014,701
11,Riders,2,2017,690


#### 索引groups

In [94]:
#索引全部groups，返回Dict
group_year.groups

#索引单个group，返回Dataframe
group_year.get_group(2014)

#索引单列
group_year['Rank'].mean()>1


#遍历groups
for name,group in group_year:
    name
    group

{2014: Int64Index([0, 2, 4, 9], dtype='int64'),
 2015: Int64Index([1, 3, 5, 10], dtype='int64'),
 2016: Int64Index([6, 8], dtype='int64'),
 2017: Int64Index([7, 11], dtype='int64')}

,Team,Rank,Points
0,Riders,1,876
2,Devils,2,863
4,Kings,3,741
9,Royals,4,701


Year
2014    True
2015    True
2016    True
2017    True
Name: Rank, dtype: bool

2014

,Team,Rank,Year,Points
0,Riders,1,2014,876
2,Devils,2,2014,863
4,Kings,3,2014,741
9,Royals,4,2014,701


...

,Team,Rank,Year,Points
1,Riders,2,2015,789
3,Devils,3,2015,673
5,kings,4,2015,812
10,Royals,1,2015,804


2016

,Team,Rank,Year,Points
6,NaN,1,2016,756
8,Riders,2,2016,694


2017

,Team,Rank,Year,Points
7,Kings,1,2017,788
11,Riders,2,2017,690


#### 常用方法

https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html

In [81]:
#对分组对象使用head函数，返回的是每个组的前几行，而不是数据集前几行
group_year.head(1)



group_year.size()

#Compute count of group, excluding missing values.
group_year.count()

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
6,NaN,1,2016,756
7,Kings,1,2017,788


Year
2014    4
2015    4
2016    2
2017    2
dtype: int64

,Team,Rank,Points
Year,,,
2014,4,4,4
2015,4,4,4
2016,1,2,2
2017,2,2,2


### Apply（SAC中的A）

#### 聚合（Aggregation）

所谓聚合就是把一堆数，变成一个标量，因此mean/sum/size/count/std/var/sem/describe/first/last/nth/min/max都是聚合函数，也可以使用自定义函数

In [103]:
group_year['Points'].agg('mean')

#同时使用多个聚合函数
group_year.agg(['sum','mean','std'])

#利用元组进行重命名
group_year.agg([('rename_sum','sum'),('rename_mean','mean')])

#指定哪些函数作用哪些列
group_year.agg({'Rank':['mean','max'],'Points':'var'})

#自定义函数
#agg函数的传入是分组逐列进行+自定义函数，就可以做许多事情
'自定义函数'
group_year['Rank'].agg(lambda x:x.max()-x.min())

Year
2014    795.25
2015    769.50
2016    725.00
2017    739.00
Name: Points, dtype: float64

Rank                Points                   
      sum mean       std    sum    mean        std
Year                                              
2014   10  2.5  1.290994   3181  795.25  87.439026
2015   10  2.5  1.290994   3078  769.50  65.035888
2016    3  1.5  0.707107   1450  725.00  43.840620
2017    3  1.5  0.707107   1478  739.00  69.296465

Rank                 Points            
     rename_sum rename_mean rename_sum rename_mean
Year                                              
2014         10         2.5       3181      795.25
2015         10         2.5       3078      769.50
2016          3         1.5       1450      725.00
2017          3         1.5       1478      739.00

Rank           Points
     mean max          var
Year                      
2014  2.5   4  7645.583333
2015  2.5   4  4229.666667
2016  1.5   2  1922.000000
2017  1.5   2  4802.000000

'自定义函数'

Year
2014    3
2015    3
2016    1
2017    1
Name: Rank, dtype: int64

#### 过滤（Filteration）

filter函数是用来筛选某些组的（务必记住结果是组的全体），传入的值应当是布尔标量

相比不分组直接过滤，区别是，如果这个组有一个不满足，整个组都被过滤掉了

In [118]:
group_year.filter(lambda x: (x['Points']> 680).all())

,Team,Rank,Year,Points
0,Riders,1,2014,876
2,Devils,2,2014,863
4,Kings,3,2014,741
6,NaN,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701
11,Riders,2,2017,690


#### 变换（Transformation）

transform传入的值是一个函数，函数作用对象是各个组的列，并且返回值需要与列长完全一致。如果返回了标量值，那么组内的所有元素会被广播为这个值。

transform与Aggregation的区别在于，transform返回一列值，Aggregation返回一个值

In [123]:
group_year.transform(lambda x:x-x.min())

group_year.transform(lambda x:x.mean())

,Rank,Points
0,0,175
1,1,116
2,1,162
3,2,0
4,2,40
5,3,139
6,0,62
7,0,98
8,1,0
9,3,0


,Rank,Points
0,2.5,795.25
1,2.5,769.50
2,2.5,795.25
3,2.5,769.50
4,2.5,795.25
5,2.5,769.50
6,1.5,725.00
7,1.5,739.00
8,1.5,725.00
9,2.5,795.25


#### 综合

apply传入的值是函数。apply函数像是其他分组函数的综合体，可以实现其他函数的所有功能。正因为如此，在所有的分组函数中，apply是应用最为广泛的。但是正因为它的灵活性，导致它的效率没有那些具有特定功能的函数的效率高，比如：transform、aggregation

In [126]:
#返回值是列表
group_year[['Rank','Points']].apply(lambda x:x-x.min())

#返回值是标量
group_year[['Rank','Points']].apply(lambda x:x.mean())

,Rank,Points
0,0,175
1,1,116
2,1,162
3,2,0
4,2,40
5,3,139
6,0,62
7,0,98
8,1,0
9,3,0


,Rank,Points
Year,,
2014,2.5,795.25
2015,2.5,769.50
2016,1.5,725.00
2017,1.5,739.00
